## Notebook to plot the results of the fitting routine tests

In [1]:
import matplotlib.pyplot as plt
import pickle, bz2
import numpy as np
from scipy.interpolate import interp1d

%matplotlib notebook

#### Setup cell

In [ ]:
dirsave = '/user/gennaro/Functional_work/Up_the_ramp_myfork/Simulations_results/'
filename = 'Test_2.pbz2'

#### Restore the svaed files and prepare all variables

In [ ]:
with bz2.BZ2File(dirsave+filename, 'rb') as f:
    dictoload = pickle.load(f)

myfluxes          =  dictoload['myfluxes']
myramps           =  dictoload['myramps']
myCRrates         =  dictoload['myCRrates']
mybgs             =  dictoload['mybgs']
j_l               =  dictoload['j_l']
k_l               =  dictoload['k_l']
goodints_l        =  dictoload['goodints_l']
CRdict_l          =  dictoload['CRdict_l']
meas_l            =  dictoload['meas_l']
counter_l         =  dictoload['counter_l']
error_l           =  dictoload['error_l']
crloops_counter_l =  dictoload['crloops_counter_l']
outerate_l        =  dictoload['outerate_l']
gof_stat_l        =  dictoload['gof_stat_l']
gof_pval_l        =  dictoload['gof_pval_l']
extra_bg_l        =  dictoload['extra_bg_l']

#Unpack the lists to regroup items by ramp

ntest = k_l[-1] + 1

gi_list  = [np.empty([ntest,mm.group_times.size-1],dtype=np.bool_) for mm in myramps]
CR_list   = [[[] for _ in range(ntest)] for mm in myramps]
meas_list = [[[] for _ in range(ntest)] for mm in myramps]
ebg_list  = [[[] for _ in range(ntest)] for mm in myramps]

counter  = np.empty([ntest,len(myfluxes)],dtype=np.int_)
error    = np.empty([ntest,len(myfluxes)],dtype=np.int_)
outerate = np.empty([ntest,len(myfluxes)])
crloops_counter  = np.empty([ntest,len(myfluxes)],dtype=np.int_)
gof_stat = np.empty([ntest,len(myfluxes)],dtype=np.int_)
gof_pval = np.empty([ntest,len(myfluxes)])

for j,k,g,C,m,c,e,cc,o,gs,gp,eb in zip(j_l,k_l,goodints_l,CRdict_l,meas_l,counter_l,error_l,crloops_counter_l,outerate_l,gof_stat_l,gof_pval_l,extra_bg_l):

    counter[k,j]  = c
    error[k,j]    = e
    crloops_counter[k,j] = cc
    outerate[k,j] = o
    gof_stat[k,j] = gs
    gof_pval[k,j] = gp

    gi_list[j][k,:] = g
    CR_list[j][k]   = C
    meas_list[j][k] = m
    ebg_list[j][k]  = eb
    

#### Regroup all the simulated CRhits and check whether they have been detected

In [ ]:
allCRtimes = []
allCRcounts = []
allCRdetect = []

ramps_with_CRs = []

for j,(myflux,myramp) in enumerate(zip(myfluxes,myramps)):

    allCRtimes_p = []
    allCRcounts_p = []
    allCRdetect_p = []
    ramps_with_CRs_p = np.zeros(len(CR_list[j]),dtype=np.bool_)
    
    CRlist_p = CR_list[j]
    for i,dd in enumerate(CRlist_p):
        if dd is not None:
            ramps_with_CRs_p[i] = True
            allCRtimes_p.extend(dd['times'])
            allCRcounts_p.extend(dd['counts'])
            for t in dd['times']:
                hit_intv_idx = np.nonzero(myramp.group_times <= t)[0][-1]
                if gi_list[j][i,hit_intv_idx] ==  False:
                    allCRdetect_p.append(True)
                else:
                    allCRdetect_p.append(False)
    
    
    #print(allCRcounts_p)
    #print(allCRtimes_p)
    allCRtimes.append(np.asarray(allCRtimes_p))
    allCRcounts.append(np.asarray(allCRcounts_p))
    allCRdetect.append(np.asarray(allCRdetect_p))
    ramps_with_CRs.append(ramps_with_CRs_p)


#### Look for false positives in the detected CRhits

In [ ]:
CR_false_positives = []

for j,(myflux,myramp) in enumerate(zip(myfluxes,myramps)):
    CR_false_positives_p = np.zeros_like(gi_list[j],dtype=np.bool_)
    print('*************')
    print(myflux,myramp.ngroups)
    for i in range(ntest):
        detected_CR_idx = np.nonzero(~gi_list[j][i,:])[0]
        ndet = len(detected_CR_idx)
        if ndet > 0:
            CRdict  = CR_list[j][i]
            for k in range(ndet):
                if CRdict is None:
                    CR_false_positives_p[i,detected_CR_idx[k]] = True
                else:   
                    ts = myramp.group_times[detected_CR_idx[k]]
                    te = myramp.group_times[detected_CR_idx[k]+1]
                    if np.all( (CRdict['times']< ts) | ((CRdict['times']> te))) == True:
                        CR_false_positives_p[i,detected_CR_idx[k]] = True
                    
    print(np.sum(CR_false_positives_p))       
    CR_false_positives.append(CR_false_positives_p)
    

### Diagnostic plots

#### Global diagnostics

In [ ]:
plt.style.use('bmh')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

In [ ]:
for j,(myflux,myramp) in enumerate(zip(myfluxes,myramps)):
    
    BM0 = (error[:,j] == 0)
    mn = np.mean(outerate[BM0,j])
    md = np.median(outerate[BM0,j])
    st = np.std(outerate[BM0,j])

    f = plt.figure(figsize=(14,8))
    
    ax1 = f.add_subplot(241)
    ax1.hist(outerate[BM0,j],bins=25)
    ax1.axvline(myflux,color='orange')
    ax1.axvline(mn,color='red')
    ax1.axvline(mn-st,color='red',linestyle='--')
    ax1.axvline(mn+st,color='red',linestyle='--')
    ax1.axvline(md,color='black')
    ax1.set_title('Fit output',fontsize=13)
    ax1.set_xlabel('e/s')

    ax2 = f.add_subplot(242)
    n,b,p = ax2.hist(counter[:,j],bins=np.linspace(1,20,39))
    n,b,p = ax2.hist(counter[BM0,j],bins=b)
    ax2.set_title('Iterations',fontsize=13)
    ax2.set_xlabel('# of loops')
    
    rCRs = ramps_with_CRs[j]
    BM2 = BM0 & (~rCRs)
    mn2 = np.mean(outerate[BM2,j])
    md2 = np.median(outerate[BM2,j])
    st2 = np.std(outerate[BM2,j])

    BM3 = BM0 & rCRs
    mn3 = np.mean(outerate[BM3,j])
    md3 = np.median(outerate[BM3,j])
    st3 = np.std(outerate[BM3,j])

    
    ax3 = f.add_subplot(243)
    if allCRcounts[j].size >0 :
        n,b,p = ax3.hist(allCRcounts[j],bins=50)
        if allCRdetect[j].size >0 :
            n,b,p = ax3.hist(allCRcounts[j][allCRdetect[j]],bins=b)
    
    ax3.axvline(4*np.sqrt(myflux*np.mean(myramp.group_times[1:]-myramp.group_times[:-1])+2*np.square(meas_list[j][0].RON_adu)),color='#22aa12',linestyle='--')
    ax3.set_title('Detected CRs',fontsize=13)
    ax3.set_xlabel('Deposited energy (DN)')
    
    ax4 = f.add_subplot(244)
    n,b,p = ax4.hist(crloops_counter[:,j],bins=np.linspace(1,10,19))
    n,b,p = ax4.hist(crloops_counter[BM0,j],bins=b)
    ax4.set_title('CR rej loops',fontsize=13)
    ax4.set_xlabel('# of loops')

    ax5 = f.add_subplot(245)
    n,b,p = ax5.hist(error[:,j],bins=np.linspace(0,4,9))
    ax5.set_title('Error message',fontsize=13) 
    ax5.set_xlabel('Error code')

    ax6 = f.add_subplot(246)   
    pthr = 0.001
    n,b,p = ax6.hist(gof_pval[:,j][np.isfinite(gof_pval[:,j])],bins=np.linspace(0,1,50))
    n,b,p = ax6.hist(gof_pval[:,j][np.isfinite(gof_pval[:,j]) & (np.asarray([kk is not None for kk in CR_list[j]]))],bins=np.linspace(0,1,50))

    ax6.set_title('Goodness of fit',fontsize=13)
    ax6.axvline(pthr,color='#22aa12',linestyle='--')
    ax6.set_xlabel('p-val')

    
    
    BMpt = gof_pval[:,j] > pthr
    
    BM1 = BM0 & BMpt
    mn1 = np.mean(outerate[BM1,j])
    md1 = np.median(outerate[BM1,j])
    st1 = np.std(outerate[BM1,j])

    
    ax7 = f.add_subplot(247)
    ax7.hist(outerate[BM1,j],bins=25)
    ax7.axvline(myflux,color='orange')
    ax7.axvline(mn1,color='red')
    ax7.axvline(mn1-st1,color='red',linestyle='--')
    ax7.axvline(mn1+st1,color='red',linestyle='--')
    ax7.axvline(md1,color='black')
    ax7.set_title('Fit output \n(p-val>thr)',fontsize=13)
    ax7.set_xlabel('e/s')

    ax8 = f.add_subplot(248)

    if mybgs[j] is not None:
        mean_bg_electron_rate = mybgs[j]['mean_bg_er']
        bg_times = mybgs[j]['times']
        bg_electron_rate = mybgs[j]['vbg_er']

        bg_int = interp1d(bg_times,bg_electron_rate,'quadratic')
        varbg = bg_int(myramps[j].read_times)

        dt = myramp.read_times[-1]-myramp.read_times[0]
        t_avg = np.trapz(varbg,myramps[j].read_times) / dt
        varbg = varbg/t_avg * mean_bg_electron_rate
        a_rms = np.sqrt(np.trapz(np.square(varbg-mean_bg_electron_rate),myramps[j].read_times)/dt)

        ax8.set_title('Background rate (e/s) \n rms/mean ={:7.4f}'.format(a_rms/mean_bg_electron_rate),fontsize=13)
        ax8.set_xlabel('s')
        ax8.plot(myramp.read_times,varbg);
        ax8.axhline(mean_bg_electron_rate,linestyle='--')
        sts = r'Flux (e/s) = {}; T$_{{exp}} = ${}; N$_{{groups}}$ = {}; Extra background (e/s) = {}'.format(myflux,myramp.group_times[-1],myramp.ngroups,mean_bg_electron_rate)
    else:
        ax8.set_title('No extra background',fontsize=13)
        sts = r'Flux (e/s) = {}; T$_{{exp}} = ${}; N$_{{groups}}$ = {}; No extra background'.format(myflux,myramp.group_times[-1],myramp.ngroups)
        
        
    for ax in [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8]:
        ax.set_axis_bgcolor('#FFFFFF')

    
    f.suptitle(sts, fontsize=14)
    f.tight_layout(rect=[0,0.03,1,.92])

    
    actual_counts = np.empty(len(meas_list[j]))
    for k in range(actual_counts.size):
        actual_counts[k] = meas_list[j][k].noiseless_counts[-1]
    
    
    exptime = myramp.group_times[-1] - myramp.group_times[0] 
    mean_signal = myflux * exptime
    poi_err = np.sqrt(mean_signal)
    act_poi_err = np.std(actual_counts)*meas_list[j][0].gain
    
    eff_RON = meas_list[j][0].effRON_e
    eff_qerr = np.sqrt(meas_list[j][0].gain/12)/meas_list[j][0].RON_e*eff_RON

    tot_noise = np.sqrt(np.sum(np.square(np.array([poi_err,eff_RON,eff_qerr]))))


    
    print('######################')
    print('Input flux (e/s):',myflux)
    print('Exposure time',exptime)
    print('Number of groups / frames / skips: {} / {} / {}'.format(myramp.ngroups,myramp.nframes,myramp.nskips))
    print('Fraction of good fits:',100.*BM0.sum().astype(np.float_)/ntest,'%')
    print(' ')    
    print('Pure Poisson / eff. RON / eff. quantization error / total relative error [% w.r.t. mean signal]: {:6.3f}, {:6.3f}, {:6.3f}, {:6.3}'.format(
        100.*poi_err/mean_signal, 100.*eff_RON/mean_signal, 100.*eff_qerr/mean_signal, 100*tot_noise/mean_signal))
    print('Pure Poisson / eff. RON / eff. quantization error / total error [e/s]: {:7.4f}, {:7.4f}, {:7.4f}, {:7.4f}'.format(
        poi_err/exptime, eff_RON/exptime, eff_qerr/exptime, tot_noise/exptime))
    print('Standard deviation from "noiseless" counts (e/s): {:7.4f}'.format(act_poi_err/exptime))
    print(' ')
    print('Output mean (e/s): {:7.4f}'.format(mn))
    print('Output median (e/s): {:7.4f}'.format(md))
    print('Output standard deviation (e/s): {:7.4f}'.format(st))
    print('Rel. % error: : {:5.2f}'.format(100.*st/myflux))
    print('Rel. % bias:: {:5.2f}'.format(100.*(mn-myflux)/myflux))
    print(' ')
    print('no-CR Output mean (e/s): {:7.4f}'.format(mn2))
    print('no-CR Output median (e/s): {:7.4f}'.format(md2))
    print('no-CR Output standard deviation (e/s): {:7.4f}'.format(st2))
    print('no-CR Rel. % error: : {:5.2f}'.format(100.*st2/myflux))
    print('no-CR Rel. % bias:: {:5.2f}'.format(100.*(mn2-myflux)/myflux))
    print(' ')
    print('yes-CR Output mean (e/s): {:7.4f}'.format(mn3))
    print('yes-CR Output median (e/s): {:7.4f}'.format(md3))
    print('yes-CR Output standard deviation (e/s): {:7.4f}'.format(st3))
    print('yes-CR Rel. % error: : {:5.2f}'.format(100.*st3/myflux))
    print('yes-CR Rel. % bias:: {:5.2f}'.format(100.*(mn3-myflux)/myflux))
    print(' ')
    print('Fraction of good fits at {:6.3f} confidence: {} %'.format(pthr,100.*np.sum(BMpt)/len(BMpt)))
    print('p-flagged Output mean (e/s) {:7.4f}'.format(mn1))
    print('p-flagged Output median (e/s) {:7.4f}'.format(md1))
    print('p-flagged Output standard deviation (e/s) {:7.4f}'.format(st1))
    print('p-flagged Rel. % error: {:5.2f}'.format(100.*st1/myflux))
    print('p-flagged Rel. % bias: {:5.2f}'.format(100.*(mn1-myflux)/myflux))
    print(' ')
    print('CR hits / detected',len(allCRcounts[j]),'/',np.sum(allCRdetect[j]))
    print('CR - false positives',np.sum(CR_false_positives[j]))
    

#### Detailed plots to examine the false positive cases

#### Detailed plots to examine the bad fit (ERR > 0) cases